# Tunga Carrefour

*Pesquisar no Carrefour os seguintes produtos* <br>

- flipper zero
- Tv box 
- bloqueador de sinal
- carregador de bateria de celular
- jammer
- reforçador de celular
- bateria de celular
- transmissor fm

A pesquisa também deve registrar a quantidade de produtos localizados em cada plataforma com a respectiva  palvra-chave

In [36]:
produtos_para_pesquisa = [
#    'flipper zero',
#     'Tv box',
#     'bloqueador de sinal',
#     'carregador de bateria de celular',
#     'jammer',
#     'reforçador de celular',
#     'bateria de celular',
     'transmissor fm'] #
# ]

### importando bibliotecas

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import urllib.parse

import pandas as pd

from time import sleep
from datetime import datetime
import re

### configurações do browser

In [18]:
#funções do webdriver
def get_driver() -> webdriver:
    """
    Retorna um objeto WebDriver para o Chrome.
    """

    # create object for chrome options

    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox') 
    chrome_options.add_argument('disable-notifications') 
    chrome_options.add_argument('--disable-infobars') 
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument("--disable-translate")
    chrome_options.add_argument("--disable-web-security")
    chrome_options.add_argument("--disable-logging")
    chrome_options.add_argument("--disable-plugins-discovery")

    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    chrome_options.add_argument(f"user-agent={user_agent}")


    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),
                                  options = chrome_options)  
    
    driver.maximize_window()# Maximiza a janela
    driver.implicitly_wait(5)
    
    return driver

def wait_for_element(driver: webdriver, element_id: str, timeout: int = 2) -> None:
    """
    Aguarda até que um elemento com o ID especificado seja carregado na página.
    """
    try:
        wait = WebDriverWait(driver, timeout)
        wait.until(EC.presence_of_element_located((By.ID, element_id)))
    except:
        pass

In [12]:
driver = get_driver() # abre o navegador
quantidade_produtos = []

# acessar carrefour

for i in produtos_para_pesquisa:

    query = urllib.parse.quote(i)
    # driver = get_driver()
    url =  f'https://www.carrefour.com.br/busca/{query}'
    driver.get(url)
    sleep(2)

    quantidade = driver.find_element(
                            By.CLASS_NAME,'vtex-search-result-3-x-totalProducts--layout'
                            ).get_attribute('innerText').split(' ')[0]
    
    # Obter a data e hora atuais
    data_hora_atual = datetime.now()
    data_hora_atual = f'{data_hora_atual.day}/{data_hora_atual.month}/{data_hora_atual.year} {data_hora_atual.hour}:{data_hora_atual.minute}:{data_hora_atual.second}'
    
    quantidade_produtos.append({'plataforma' :'Carrefour',
                                'produto'    : i,
                                'quantidade' : quantidade,
                                'data_coleta': data_hora_atual
                               })

driver.quit()

### salvar as quantidades

In [8]:
df = pd.DataFrame(quantidade_produtos)
df.to_csv('quantidades_carrefour.csv', index = None, sep = ';')
display(df)

,plataforma,produto,quantidade,data_coleta
0,Carrefour,flipper zero,2,6/6/2023 14:32:5
1,Carrefour,Tv box,20,6/6/2023 14:32:16
2,Carrefour,bloqueador de sinal,2,6/6/2023 14:32:26
3,Carrefour,carregador de bateria de celular,29,6/6/2023 14:32:36
4,Carrefour,jammer,2,6/6/2023 14:32:47
5,Carrefour,reforçador de celular,409,6/6/2023 14:33:0
6,Carrefour,bateria de celular,284,6/6/2023 14:33:12
7,Carrefour,transmissor fm,78,6/6/2023 14:33:23


### capturar os links dos produtos

In [39]:
driver = get_driver() # abre o navegador
driver.get('https://www.carrefour.com.br/busca/transmissor%20fm')

In [40]:
try:
    # <a href="/busca/transmissor%20fm?page=6" class="carrefourbr-carrefour-components-0-x-Pagination_item_Link">6</a>
    ultima_pagina = driver.find_elements(By.CLASS_NAME, 'carrefourbr-carrefour-components-0-x-Pagination_ItemsList')
    ultima_pagina = ultima_pagina[0].find_elements(By.TAG_NAME, 'li')
    ultima_pagina = int(ultima_pagina[-2].text.strip())

except:
    ultima_pagina = 1
        
ultima_pagina

6

In [42]:
links_produtos = set()

# driver = get_driver() # abre o navegador

for produto_pesquisado in produtos_para_pesquisa:

    query = urllib.parse.quote(produto_pesquisado)
    # driver = get_driver()
    url =  f'https://www.carrefour.com.br/busca/{query}'
    driver.get(url)
    sleep(4)

    ### descobrir a última página

    try:
        #<a href="/busca/transmissor%20fm?page=6" class="carrefourbr-carrefour-components-0-x-Pagination_item_Link">6</a>
        ultima_pagina = driver.find_elements(By.CLASS_NAME, 'carrefourbr-carrefour-components-0-x-Pagination_ItemsList')
        ultima_pagina = ultima_pagina[0].find_elements(By.TAG_NAME, 'li')
        ultima_pagina = int(ultima_pagina[-2].text.strip())

    except:
        ultima_pagina = 1

    for pagina in range(ultima_pagina):

        try:
            print('pesquisando página ', pagina+1, 'de', ultima_pagina)
            wait_for_element(driver, 'styles_iconpack', 5 )

            a_elements = driver.find_elements(By.TAG_NAME, 'a')
            for element in a_elements:

                if element.get_attribute('href')[-2:] == '/p':
                    links_produtos.add(element.get_attribute('href'))

            if pagina < ultima_pagina-1:
                driver.get(f'https://www.carrefour.com.br/busca/{query}?page={pagina+2}')

        except Exception as e:
            print(e)
    
driver.quit()

pesquisando página  1 de 6
pesquisando página  2 de 6
pesquisando página  3 de 6
pesquisando página  4 de 6
pesquisando página  5 de 6
Message: script timeout
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x007FA813+48355]
	(No symbol) [0x0078C4B1]
	(No symbol) [0x00695220]
	(No symbol) [0x006ED2A3]
	(No symbol) [0x006DA73C]
	(No symbol) [0x006EC922]
	(No symbol) [0x006DA536]
	(No symbol) [0x006B82DC]
	(No symbol) [0x006B93DD]
	GetHandleVerifier [0x00A5AABD+2539405]
	GetHandleVerifier [0x00A9A78F+2800735]
	GetHandleVerifier [0x00A9456C+2775612]
	GetHandleVerifier [0x008851E0+616112]
	(No symbol) [0x00795F8C]
	(No symbol) [0x00792328]
	(No symbol) [0x0079240B]
	(No symbol) [0x00784FF7]
	BaseThreadInitThunk [0x75D36359+25]
	RtlGetAppContainerNamedObjectPath [0x77967B74+228]
	RtlGetAppContainerNamedObjectPath [0x77967B44+180]

pesquisando página  6 de 6


In [43]:
print(len(links_produtos))

75


In [44]:
links_produtos = list(links_produtos)
links_produtos[0]

'https://www.carrefour.com.br/transmissor-bluetooth-fm-imden-v50-para-carro-qc30typec-pd-18w-wireless-bluetooth-fm-radio-adapter-b-mp922773539/p'

### navegar pelos produtos

In [45]:
driver = get_driver() # abre o navegador
tempo_de_espera = 3
falhas = []
produtos = []
final = len(links_produtos)

for link in links_produtos:

    final = final -1
    if final % 10 == 0:
        print(final)
        
    try:
        driver.get(link)
        sleep(tempo_de_espera)



        try:
            titulo_produto = driver.find_element(By.TAG_NAME, 'h1').text
        except:
            titulo_produto = ''

        try:
            categorias = driver.find_elements(By.CLASS_NAME, 'vtex-breadcrumb-1-x-container')[0].text
            categorias.split('\n')[0:-1]
        except:
            categorias = ''

        try:
            id_vendedor    = driver.find_element(By.XPATH, '//*[@id="seller_info"]/div/a').get_attribute('href')
            id_vendedor      = id_vendedor.split('/')[-1]
        except:
            id_vendedor    = ''

        try:        
            preco = driver.find_element(By.CLASS_NAME, 'andes-money-amount__fraction').text
            try:
                preco = preco + ',' + driver.find_element(By.CLASS_NAME, 'andes-money-amount__cents').text
            except:
                pass

        except:
            preco = ''

        try:
            descricao      = driver.find_element(By.CLASS_NAME, 'ui-pdp-description__content').text
        except:
            descricao      = ''

        quantidade = 1 

        try:
            detalhes_produto = driver.find_elements(By.CLASS_NAME, 
                                'vtex-flex-layout-0-x-flexCol--product-details-content'
                                              )[0].get_attribute('innerText')
        except:
            detalhes_produto = ''

        try:
            marca = driver.find_element(By.CLASS_NAME, 'vtex-store-components-3-x-productBrandName').get_attribute('innerText')
        except:
            marca = ''

        modelo = ''
        localizacao = ''

        #     temp_dict = {
        #         'produto_pesquisado' : chave,
        #         'url' : item,
        #         'titulo_produto' : titulo_produto,
        #         # 'marca' : marca,
        #         'id_vendedor': id_vendedor,
        #         'preco': preco,
        #         'quantidade': quantidade,
        #         'descricao': descricao ,
        #         'homologado': '' #,
        #         # modelo' : modelo

        #     }


        produto = {
            'plataforma'        : 'Carrefour',
            'produto_pesquisado': 'transmissor fm',
            'titulo_produto'    : titulo_produto,
            'detalhes_produto'  : detalhes_produto,
            'marca'             : marca,
            'modelo'            : modelo,
            'id_vendedor'       : id_vendedor,
            'localizacao'       : localizacao,
            'preco_minimo'      : preco,
            'preco_maximo'      : '',
            'preco_promo_minimo': '',
            'preco_promo_maximo': '',
            'quantidade'        : quantidade,
            'descricao'         : descricao,
            'categorias'         : categorias,
            'homologado'        : '',
            'vendas_anteriores' : '',
            'url'               : link                                            
        }        


        produtos.append(produto)        
        
    except Exception as e:
        falhas.append(link)
        print(e)

driver.quit()

70
60
50
40
30
20
10
0


In [46]:
len(produtos)

75

In [47]:
len(falhas)

0

In [48]:
df = pd.DataFrame(produtos)
df.to_csv(f'resultados_carrefour_transmissor_fm.csv', encoding='utf-8', index = False, sep =';')

In [49]:
df

,plataforma,produto_pesquisado,titulo_produto,detalhes_produto,marca,modelo,id_vendedor,localizacao,preco_minimo,preco_maximo,preco_promo_minimo,preco_promo_maximo,quantidade,descricao,categorias,homologado,vendas_anteriores,url
0,Carrefour,transmissor fm,Transmissor Bluetooth Fm Imden V5.0 Para Carro...,Especificações técnicas\nCaracterísticas do Pr...,,,,,,,,,1,,TVs e Vídeo\nAcessórios de TVs e Vídeo\nTransm...,,,https://www.carrefour.com.br/transmissor-bluet...
1,Carrefour,transmissor fm,Fone De Ouvido Sem Fio Fm Profissional Para Fo...,Especificações técnicas\nVariantes\nVariante_C...,,,,,,,,,1,,Informática\nAcessórios para Informática\nFone...,,,https://www.carrefour.com.br/fone-de-ouvido-se...
2,Carrefour,transmissor fm,"Transmissor Bluetooth Fm Veicular Usb Aux, Dee...",Especificações técnicas\nVariantes\nVariante_C...,,,,,,,,,1,,Automotivo\nAutopeças\nKit de Transmissão Veic...,,,https://www.carrefour.com.br/transmissor-bluet...
3,Carrefour,transmissor fm,Transmissor FM Veicular Bluetooth USB com Disp...,Especificações técnicas\nCaracterísticas do Pr...,,,,,,,,,1,,Áudio\nAcessórios para Áudio Profissional\nTra...,,,https://www.carrefour.com.br/transmissor-fm-ve...
4,Carrefour,transmissor fm,"Transmissor Orikgher Bluetooth Fm Para Carro, ...",Especificações técnicas\nDimensões e Peso\nDim...,,,,,,,,,1,,Informática\nAcessórios para Informática\nExte...,,,https://www.carrefour.com.br/transmissor-orikg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Carrefour,transmissor fm,Transmissor Veicular Com Bluetooth Fm Mp3 Usb ...,Especificações técnicas\nImportante\nCódigo da...,,,,,,,,,1,,Automotivo\nAutopeças\nKit de Transmissão Veic...,,,https://www.carrefour.com.br/transmissor-veicu...
71,Carrefour,transmissor fm,Hmwy-car Bluetooth Fm Transmissor Mp3 Player M...,Especificações técnicas\nVariantes\nVariante_C...,,,,,,,,,1,,"Casa, Construção e Ferramentas\nElétrica e Hid...",,,https://www.carrefour.com.br/hmwycar-bluetooth...
72,Carrefour,transmissor fm,Transmissor Fm Veicular Bluetooth Usb Com Disp...,Descrição do produto\nTransmissor Fm Veicular ...,,,,,,,,,1,,Áudio\nAcessórios para Áudio Profissional\nTra...,,,https://www.carrefour.com.br/transmissor-fm-ve...
73,Carrefour,transmissor fm,Mp3 Car Lcd Sem Fio Bluetooth Transmissor Fm E...,Especificações técnicas\nVariantes\nVariante_C...,,,,,,,,,1,,Automotivo\nAutopeças\nKit de Transmissão Veic...,,,https://www.carrefour.com.br/mp3-car-lcd-sem-f...


In [35]:


try:
    titulo_produto = driver.find_element(By.TAG_NAME, 'h1').text
except:
    titulo_produto = ''

try:
    categorias = driver.find_elements(By.CLASS_NAME, 'vtex-breadcrumb-1-x-container')[0].text
    categorias.split('\n')[0:-1]
except:
    categorias = ''

try:
    id_vendedor    = driver.find_element(By.XPATH, '//*[@id="seller_info"]/div/a').get_attribute('href')
    id_vendedor      = id_vendedor.split('/')[-1]
except:
    id_vendedor    = ''

try:        
    preco = driver.find_element(By.CLASS_NAME, 'andes-money-amount__fraction').text
    try:
        preco = preco + ',' + driver.find_element(By.CLASS_NAME, 'andes-money-amount__cents').text
    except:
        pass

except:
    preco = ''

try:
    descricao      = driver.find_element(By.CLASS_NAME, 'ui-pdp-description__content').text
except:
    descricao      = ''

quantidade = 1 

try:
    detalhes_produto = driver.find_elements(By.CLASS_NAME, 
                        'vtex-flex-layout-0-x-flexCol--product-details-content'
                                      )[0].get_attribute('innerText')
except:
    detalhes_produto = ''
    
try:
    marca = driver.find_element(By.CLASS_NAME, 'vtex-store-components-3-x-productBrandName').get_attribute('innerText')
except:
    marca = ''

modelo = ''
localizacao = ''

#     temp_dict = {
#         'produto_pesquisado' : chave,
#         'url' : item,
#         'titulo_produto' : titulo_produto,
#         # 'marca' : marca,
#         'id_vendedor': id_vendedor,
#         'preco': preco,
#         'quantidade': quantidade,
#         'descricao': descricao ,
#         'homologado': '' #,
#         # modelo' : modelo

#     }


produto = {
    'plataforma'        : 'Carrefour',
    'produto_pesquisado': 'flipper zero',
    'titulo_produto'    : titulo_produto,
    'detalhes_produto'  : detalhes_produto,
    'marca'             : marca,
    'modelo'            : modelo,
    'id_vendedor'       : id_vendedor,
    'localizacao'       : localizacao,
    'preco_minimo'      : preco,
    'preco_maximo'      : '',
    'preco_promo_minimo': '',
    'preco_promo_maximo': '',
    'quantidade'        : quantidade,
    'descricao'         : descricao,
    'homologado'        : '',
    'vendas_anteriores' : '',
    'url'               : link                                            
}        
      


produto

{'plataforma': 'Carrefour',
 'produto_pesquisado': 'flipper zero',
 'titulo_produto': 'Smartphone Apple iPhone 13 Pro, 256GB, Prateado, 5G, 6.1" Super Retina XDR OLED, Câmera Tripla 12MP, Selfie 12MP, IOS 15',
 'detalhes_produto': 'Especificações técnicas\nVariantes\nVariante\nCor\nVariante_Cor\nPrata\nCaracterísticas do Produto\nOperadora\nDesbloqueado\nTecnologia\n"Rede LTE de classe Gigabit com MIMO 2x2 e LAA Wi-Fi 6 802.11ax com MIMO 2x2 Bluetooth 5.0 "\nRecursos da Câmera\n"Câmera Sistema de câmera Pro de 12MP (novas câmeras teleobjetiva, grande-angular e ultra-angular) Teleobjetiva: abertura ƒ/2.8 Grande-angular: abertura ƒ/1.5 Ultra-angular: abertura ƒ/1.8 e ângulo de visão de 120° Zoom in óptico de 3x, zoom out óptico de 2x; alcance de zoom óptico de 6x Zoom digital até 15x Retratos com modo Noite possibilitados pelo Scanner LiDAR Modo Retrato com efeito bokeh avançado e Controle de Profundidade Iluminação de Retrato com seis efeitos (Luz Natural, Luz de Estúdio, Luz de Contorn

In [31]:
"""
<span class="vtex-store-components-3-x-productBrandName">Apple</span>
"""


marca = driver.find_element(By.CLASS_NAME, 'vtex-store-components-3-x-productBrandName').get_attribute('innerText')
print(marca)

Apple


In [29]:
"""
<div class="vtex-flex-layout-0-x-flexCol vtex-flex-layout-0-x-flexCol--product-details-content  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--product-details-content pb0" style="height: 100%;"><nav class="vtex-menu-2-x-menuContainerNav vtex-menu-2-x-menuContainerNav--gato-anchor"><ul class="vtex-menu-2-x-menuContainer vtex-menu-2-x-menuContainer--gato-anchor list flex pl0 mv0 flex-row"><li class="vtex-menu-2-x-menuItem list vtex-menu-2-x-menuItem vtex-menu-2-x-menuItem--isClosed"><div class="vtex-menu-2-x-styledLinkContainer mh6 pv5"><a href="#product-specification-anchor" id="product-specification-anchor" title="" class="vtex-menu-2-x-styledLink no-underline pointer t-body c-on-base pointer"><div class="vtex-menu-2-x-styledLinkContent flex justify-between nowrap"></div></a></div></li></ul></nav></div><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--product-details-content pb0" style="height: 100%;"><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--product-custom-content"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--product-custom-content items-stretch w-100"><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 100%;"><div class="vtex-store-components-3-x-specificationsTableContainer vtex-store-components-3-x-specificationsTableContainer--pdp-details mt9 mt0-l pl8-l"><h2 class="vtex-store-components-3-x-specificationsTitle vtex-store-components-3-x-specificationsTitle--pdp-details t-heading-5 mb5 mt0"><div style="display: contents;">Especificações técnicas</div></h2><table class="vtex-store-components-3-x-specificationsTable vtex-store-components-3-x-specificationsTable--pdp-details w-100 bg-base border-collapse"><thead class="vtex-store-components-3-x-specificationsTableHead vtex-store-components-3-x-specificationsTableHead--pdp-details"><tr class="vtex-store-components-3-x-specificationsTableRow vtex-store-components-3-x-specificationsTableRow--pdp-details"><th class="vtex-store-components-3-x-specificationsTablePropertyHeading vtex-store-components-3-x-specificationsTablePropertyHeading--pdp-details w-50 b--muted-4 bb bt c-muted-2 t-body tl pa5">Propriedade</th><th class="vtex-store-components-3-x-specificationsTableSpecificationHeading vtex-store-components-3-x-specificationsTableSpecificationHeading--pdp-details w-50 b--muted-4 bb bt c-muted-2 t-body tl pa5">Especificação</th></tr></thead><tbody class="vtex-store-components-3-x-specificationsTableBody vtex-store-components-3-x-specificationsTableBody--pdp-details"></tbody></table></div></div></div></div></div><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--product-details-content pb0" style="height: 100%;"><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--productSpecificationTitle"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--productSpecificationTitle items-stretch w-100"></div></div></div><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--product-details-content pb0" style="height: 100%;"><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--standout-content"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--standout-content items-stretch w-100"><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 100%;"><div id="standoutDivAutomatico"></div></div></div></div></div><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--product-details-content pb0" style="height: 100%;"><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--productSpecificationGroup"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--productSpecificationGroup items-stretch w-100"><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 100%;"><div class="vtex-flex-layout-0-x-flexCol  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild pb0" style="height: 100%;"><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--productSpecificationGroupName"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--productSpecificationGroupName items-stretch w-100"><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 100%;"><span data-specification-group="Dimensões e Peso" class="vtex-product-specifications-1-x-groupName">Dimensões e Peso</span></div></div></div></div><div class="vtex-flex-layout-0-x-flexColChild pb0" style="height: 100%;"><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--productSpecification"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--productSpecification items-stretch w-100"><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 25%;"><div class="vtex-flex-layout-0-x-flexCol vtex-flex-layout-0-x-flexCol--productSpecificationName  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--productSpecificationName pb0" style="height: 100%;"><span data-specification-group="Dimensões e Peso" data-specification-name="Dimensões aproximadas do produto (AxLxP)" class="vtex-product-specifications-1-x-specificationName">Dimensões aproximadas do produto (AxLxP)</span></div></div></div><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 75%;"><div class="vtex-flex-layout-0-x-flexCol vtex-flex-layout-0-x-flexCol--productSpecificationValue  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--productSpecificationValue pb0" style="height: 100%;"><span data-specification-group="Dimensões e Peso" data-specification-name="Dimensões aproximadas do produto (AxLxP)" data-specification-value="5x9x19" class="vtex-product-specifications-1-x-specificationValue vtex-product-specifications-1-x-specificationValue--first vtex-product-specifications-1-x-specificationValue--last">5x9x19</span></div></div></div></div></div><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--productSpecification"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--productSpecification items-stretch w-100"><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 25%;"><div class="vtex-flex-layout-0-x-flexCol vtex-flex-layout-0-x-flexCol--productSpecificationName  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--productSpecificationName pb0" style="height: 100%;"><span data-specification-group="Dimensões e Peso" data-specification-name="Peso aproximado do produto sem embalagem" class="vtex-product-specifications-1-x-specificationName">Peso aproximado do produto sem embalagem</span></div></div></div><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 75%;"><div class="vtex-flex-layout-0-x-flexCol vtex-flex-layout-0-x-flexCol--productSpecificationValue  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--productSpecificationValue pb0" style="height: 100%;"><span data-specification-group="Dimensões e Peso" data-specification-name="Peso aproximado do produto sem embalagem" data-specification-value="408" class="vtex-product-specifications-1-x-specificationValue vtex-product-specifications-1-x-specificationValue--first vtex-product-specifications-1-x-specificationValue--last">408</span></div></div></div></div></div></div></div></div></div></div><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--productSpecificationGroup"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--productSpecificationGroup items-stretch w-100"><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 100%;"><div class="vtex-flex-layout-0-x-flexCol  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild pb0" style="height: 100%;"><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--productSpecificationGroupName"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--productSpecificationGroupName items-stretch w-100"><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 100%;"><span data-specification-group="Importante" class="vtex-product-specifications-1-x-groupName">Importante</span></div></div></div></div><div class="vtex-flex-layout-0-x-flexColChild pb0" style="height: 100%;"><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--productSpecification"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--productSpecification items-stretch w-100"><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 25%;"><div class="vtex-flex-layout-0-x-flexCol vtex-flex-layout-0-x-flexCol--productSpecificationName  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--productSpecificationName pb0" style="height: 100%;"><span data-specification-group="Importante" data-specification-name="SAC do Fornecedor" class="vtex-product-specifications-1-x-specificationName">SAC do Fornecedor</span></div></div></div><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 75%;"><div class="vtex-flex-layout-0-x-flexCol vtex-flex-layout-0-x-flexCol--productSpecificationValue  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--productSpecificationValue pb0" style="height: 100%;"><span data-specification-group="Importante" data-specification-name="SAC do Fornecedor" data-specification-value="info@nocnocgroup.com" class="vtex-product-specifications-1-x-specificationValue vtex-product-specifications-1-x-specificationValue--first vtex-product-specifications-1-x-specificationValue--last">info@nocnocgroup.com</span></div></div></div></div></div><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--productSpecification"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--productSpecification items-stretch w-100"><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 25%;"><div class="vtex-flex-layout-0-x-flexCol vtex-flex-layout-0-x-flexCol--productSpecificationName  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--productSpecificationName pb0" style="height: 100%;"><span data-specification-group="Importante" data-specification-name="Garantia do Fornecedor" class="vtex-product-specifications-1-x-specificationName">Garantia do Fornecedor</span></div></div></div><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 75%;"><div class="vtex-flex-layout-0-x-flexCol vtex-flex-layout-0-x-flexCol--productSpecificationValue  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--productSpecificationValue pb0" style="height: 100%;"><span data-specification-group="Importante" data-specification-name="Garantia do Fornecedor" data-specification-value="0" class="vtex-product-specifications-1-x-specificationValue vtex-product-specifications-1-x-specificationValue--first vtex-product-specifications-1-x-specificationValue--last">0</span></div></div></div></div></div><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--productSpecification"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--productSpecification items-stretch w-100"><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 25%;"><div class="vtex-flex-layout-0-x-flexCol vtex-flex-layout-0-x-flexCol--productSpecificationName  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--productSpecificationName pb0" style="height: 100%;"><span data-specification-group="Importante" data-specification-name="Produto Internacional" class="vtex-product-specifications-1-x-specificationName">Produto Internacional</span></div></div></div><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 75%;"><div class="vtex-flex-layout-0-x-flexCol vtex-flex-layout-0-x-flexCol--productSpecificationValue  ml0 mr0 pl0 pr0      flex flex-column h-100 w-100"><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--productSpecificationValue pb0" style="height: 100%;"><span data-specification-group="Importante" data-specification-name="Produto Internacional" data-specification-value="Sim" class="vtex-product-specifications-1-x-specificationValue vtex-product-specifications-1-x-specificationValue--first vtex-product-specifications-1-x-specificationValue--last">Sim</span></div></div></div></div></div></div></div></div></div></div></div><div class="vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--product-details-content pb0" style="height: 100%;"><div class="vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--product-description"><div class="flex mt0 mb0 pt0 pb0    justify-start vtex-flex-layout-0-x-flexRowContent vtex-flex-layout-0-x-flexRowContent--product-description items-stretch w-100"><div class="pr0 items-stretch vtex-flex-layout-0-x-stretchChildrenWidth   flex" style="width: 100%;"><div class="vtex-store-components-3-x-productDescriptionContainer"><h2 class="vtex-store-components-3-x-productDescriptionTitle t-heading-5 mb5 mt0">Descrição do produto</h2><div class="vtex-store-components-3-x-productDescriptionText c-muted-1"><div style="display: contents;">Resuxi Cordões De Cabelo Sem Fio Para Homens Zero Gapped T-blade Trimmer, Aparador De Cabelo Profissional Edgemers Barber Clippers Para Corte De Cabelo, Raspadores De Aparador De Barba Detalhe Para Homens Com Cavaleiro De Ouro</div></div></div></div></div></div></div></div>
"""

detalhes_produto = driver.find_elements(By.CLASS_NAME, 
                        'vtex-flex-layout-0-x-flexCol--product-details-content'
                                      )[0].get_attribute('innerText')
print(detalhes_produto)

Especificações técnicas
Dimensões e Peso
Dimensões aproximadas do produto (AxLxP)
5x9x19
Peso aproximado do produto sem embalagem
408
Importante
SAC do Fornecedor
info@nocnocgroup.com
Garantia do Fornecedor
0
Produto Internacional
Sim
Descrição do produto
Resuxi Cordões De Cabelo Sem Fio Para Homens Zero Gapped T-blade Trimmer, Aparador De Cabelo Profissional Edgemers Barber Clippers Para Corte De Cabelo, Raspadores De Aparador De Barba Detalhe Para Homens Com Cavaleiro De Ouro


In [142]:
links = set()
query = 'tv box'
query = urllib.parse.quote(query)
driver = get_driver()
navigate_to_page(driver, f'https://www.carrefour.com.br/busca/{query}')
sleep(3)

### descobrir a última página

try:
    ultima_pagina = driver.find_elements(By.CLASS_NAME, 'carrefourbr-carrefour-components-0-x-Pagination_ItemsList')
    ultima_pagina = ultima_pagina[0].find_elements(By.TAG_NAME, 'a')
    ultima_pagina = ultima_pagina[-1].get_attribute('href') 
    ultima_pagina = int(ultima_pagina[ultima_pagina.find('=')+1:])

except:
    ultima_pagina = 1

for pagina in range(ultima_pagina):
    try:
        print('pesquisando página ', pagina+1, 'de', ultima_pagina)
        wait_for_element(driver, 'styles_iconpack', 5 )
        sleep(3)

        #descobrir os links dos produtos
        # grid_produtos = driver.find_elements(By.CLASS_NAME, 'carrefourbr-carrefour-components-0-x-gallery')
        # a_elements = grid_produtos[0].find_elements(By.TAG_NAME, 'a')
        a_elements = driver.find_elements(By.TAG_NAME, 'a')
        for element in a_elements:

            if element.get_attribute('href')[-2:] == '/p':
                # print(element.get_attribute('href'))
                links.add(element.get_attribute('href'))

        if pagina < ultima_pagina-1:
            navigate_to_page(driver, f'https://www.carrefour.com.br/busca/{query}?page={pagina+2}')
            
    except:
        pass


pesquisando página  1 de 2
558
pagina 0
pesquisando página  2 de 2
559


In [143]:
len(links)

21

In [ ]:



def element_exists(driver: webdriver, element_id: str) -> bool:
    """
    Verifica se um elemento com o ID especificado existe na página.
    """
    try:
        driver.find_element(By.ID, element_id)
        return True
    except:
        return False


def wait_for_element(driver: webdriver, element_id: str, timeout: int = 2) -> None:
    """
    Aguarda até que um elemento com o ID especificado seja carregado na página.
    """
    wait = WebDriverWait(driver, timeout)
    wait.until(EC.presence_of_element_located((By.ID, element_id)))
        
def get_product_details(dicionario, driver):

    """
    1. Navega pelos links dos produtos coletados na varredura inicial

    captura os atributos dos produtos grava em um dataframe
    
    Args:
        dicionario (dict): O dicionário dos produtos coletados na varredura inicial 
    
    Returns:
        Um dataframe pandas com as seguintes colunas:
        
        produto_pesquisado | # marca | url | titulo_produto | id_vendedor |
        preco | quantidade | descricao | # modelo
        
    """
    # driver = get_driver() # abre o navegador
    # tempo_de_espera = 1     # tempo de espera em segundos para carregar
    
    df = pd.DataFrame()
    for chave, valor in dicionario.items():

        for item in dicionario[chave]:
            
            # navega até a página do produto
            driver.get(item)
            sleep(tempo_de_espera)
            
            try:
                titulo_produto = driver.find_element(By.CLASS_NAME, 'ui-pdp-title').text
            except:
                titulo_produto = ''

            try:
                id_vendedor    = driver.find_element(By.XPATH, '//*[@id="seller_info"]/div/a').get_attribute('href')
                id_vendedor      = id_vendedor.split('/')[-1]
            except:
                id_vendedor    = ''

            try:        
                preco = driver.find_element(By.CLASS_NAME, 'andes-money-amount__fraction').text
                try:
                    preco = preco + ',' + driver.find_element(By.CLASS_NAME, 'andes-money-amount__cents').text
                except:
                    pass

            except:
                preco = ''
                
            try:
                descricao      = driver.find_element(By.CLASS_NAME, 'ui-pdp-description__content').text[0:500]
            except:
                descricao      = ''
            try:
                quantidade     = driver.find_element(By.CLASS_NAME, 'ui-pdp-buybox__quantity__available').text
                padrao = r'\((\d+)\s+\w+\)'
                match = re.search(padrao, quantidade)

                if match:
                    quantidade = int(match.group(1))

                else:
                    quantidade = 1 
            except:
                quantidade     = -1 
            
            
            
            temp_dict = {
                'produto_pesquisado' : chave,
                'url' : item,
                'titulo_produto' : titulo_produto,
                # 'marca' : marca,
                'id_vendedor': id_vendedor,
                'preco': preco,
                'quantidade': quantidade,
                'descricao': descricao ,
                'homologado': '' #,
                # modelo' : modelo

            }

#         # Colocar aqui a função de testar as yes-words e no-words
            
            df = pd.concat([df, pd.DataFrame([temp_dict])], ignore_index=True)
            # df = df.append(temp_dict, ignore_index = True)

    return df    
